# Inspecting a LAS/LAZ file

In [ ]:
pdal --version

In [ ]:
ls -l ./data

In general, it is best to use absolute paths to specify paths to datafiles.  Relative paths may cause issues.

In [ ]:
pwd

In [ ]:
laz_file=$PWD'/data/FoxIsland.laz'
CWD=`pwd`
echo $laz_file
echo $CWD

## Metadata

In [ ]:
pdal info --metadata $laz_file

To get the schema for a dataset:

In [ ]:
pdal info --schema ./data/FoxIsland.laz

To get an overall metadata summary for a dataset:

In [ ]:
pdal info $laz_file --summary

## Printing Points

Print the values for the first point in the file:

In [ ]:
pdal info $laz_file -p 0

Print the values for the first 5 points in the file

In [ ]:
pdal info $laz_file -p 0-10

Print the first 10 points, and extract only the elevation values using bash commands

In [ ]:
pdal info $laz_file -p 0-10|grep  "Z"|awk '{print $2}' FS=': '

jq is a command line JSON parser, and can be a useful tool for these types of operations. Note if returning more than one point, specify "[]" to return all the points in the array.

In [ ]:
pdal info $laz_file -p 0-10|jq -r .points.point[].Z

## Attribute Statistics

Use the [info application](https://pdal.io/en/2.4.3/apps/info.html) with the --stats flag and perform filtering to get a summary of the classifications for a given lidar file.

In [ ]:
pdal info ./data/FoxIsland.laz --stats --filters.stats.dimensions=Classification  --filters.stats.count=Classification

Perform both a metadata dump and count of Classification and ReturnNumber and output to a JSON file:

In [ ]:
pdal info --metadata --stats --filters.stats.dimensions=ReturnNumber,Classification --filters.stats.count=ReturnNumber,Classification ./data/FoxIsland.laz > ./data/fileinfo.json

## Reprojection

In [ ]:
ls

Run a pipeline to project a file from Oregon Lambert to Geographic Lat/Lon coordindates:

In [ ]:
pdal pipeline ./pipelines/Reproject_Ex.json

In [ ]:
ls ./data

Inspect the coordinates to make sure the xy values make sense..

In [ ]:
pdal info --metadata ./data/FoxIsland_4326.laz

# Tiling data

In [ ]:
pdal pipeline ./pipelines/TileLAZ.json

# Thinning data

Point cloud files can often be quite large and cumbersome to work with.  Depending on the objective, it is often useful to thin a dataset in order to make it easier and faster to work with. The [filters.sample](https://pdal.io/en/2.5.3/stages/filters.sample.html#filters-sample) utilizes a Poisson sampling to thin the dataset.

In [ ]:
pdal translate ./data/FoxIsland.laz ./data/FoxIsland_Thin1m.laz sample --filters.sample.radius=1

## Calculating a Boundary

Utilizing the info command, the boundary of a dataset can be obtained by using the "--boundary" flag.  This will output the boundary in WKT format in JSON-formatted output.

In [ ]:
ls

Use the "--boundary" flag to get the boundary of data over the Siuslaw River:

In [ ]:
pdal info ./data/Siuslaw.laz --boundary 

However, to get a boundary in vector format to visualize in a GIS or Google Earth requires some additional steps.  The PDAL command, [tindex](https://pdal.io/en/2.5.3/apps/tindex.html#tindex-command) is used to create a boundary that utilizes the [hexbin filter](https://pdal.io/en/2.5.3/stages/filters.hexbin.html#filters-hexbin)

To get a basic boundary:

In [ ]:
pdal tindex create --tindex ./data/Siuslaw_bounds.shp --filespec ./data/Siuslaw.laz -f "ESRI Shapefile"

Load the shapefile into a GIS to see its extent:
![Siuslaw Bounds](./images/SiuslawRiverBounds.jpeg)

For rough estimations of boundaries, this is usually sufficient. To obtain a more precise fit of the data alter some of the parameters in the [filters.hexbin](https://pdal.io/en/2.5.3/stages/filters.hexbin.html#filters-hexbin) command.  The "edge_size" parameter is particularly useful for this scenario as it controls the size of the hexagon boundaries used to estimate whether a section of the dataset should be considered. Finding an appropriate value for edge_size can be an iterative process.  For example, try using a vlue of "50" units (for this dataset, the units are feet).

In [ ]:
pdal tindex create --tindex ./data/Siuslaw_bounds50.shp --filters.hexbin.edge_size=50 --filespec ./data/Siuslaw.laz -f "ESRI Shapefile"

Load this version of the boundary into a GIS:
![Siuslaw Bounds_Fit](./images/SiuslawRiverBounds_Fit.png)

Note how this is a much better fit to the data, and shows regions where there is no data (over the water). However, use caution with the edge_size parameter as setting it too low might not capture an appropriate amount of data, and mis-represent the data coverage.

### Visualizing Point Density

In [ ]:
pdal density ./data/Siuslaw.laz -o ./data/Siuslaw_density.shp -f "ESRI Shapefile"

Size of the hexagons can be controlled with the optional parameter, "--filters.hexbin.edge_size"

In [ ]:
pdal density --filters.hexbin.edge_size=50 ./data/Siuslaw.laz -o ./data/Siuslaw_density50.shp -f "ESRI Shapefile"

## Filtering Noise

In [ ]:
pdal pipeline ./pipelines/NoiseFilter.json

In [ ]:
pdal info ./data/FoxIsland_Clean.laz --stats --filters.stats.dimensions=Classification  --filters.stats.count=Classification

# Cropping Data

[filters.crop](https://pdal.io/en/2.5.3/stages/filters.crop.html) removes points that fall outside or inside a cropping bounding box. The “polygon” option takes a WKT-formatted string to apply the clipping mask.

In [ ]:
pdal pipeline ./pipelines/Clip_ex.json

# Ground Classifications

In [ ]:
pdal pipeline ./pipelines/ExtractGround.json

In [ ]:
pdal pipeline ./pipelines/CreateGround.json

# Create Rasters

In [ ]:
pdal pipeline ./pipelines/CreateDSM.json

In [ ]:
pdal pipeline ./pipelines/CreateDTM.json

## Dealing with NoData Values

- Note that when calculating the min value with a smaller cell size for a ground-classified data, there may be gaps in the data.  GDAL has a useful tool called, [gdal_fillnodata.py](https://gdal.org/programs/gdal_fillnodata.html) to fill in gaps in datasets by interpolating data from the edges of missing areas.



In [ ]:
gdal_fillnodata.py -si 2 ./data/FoxIsland_DTM.tif ./data/FoxIsland_DTM_Fill.tif

- By default the algorithm uses a 100 pixel distance to search for pixel values to use with the interpolation, but this distance can be customized with the "-md" option.  
- The -si option is the number of times to run a 3x3 averaging filter over the interpolated area to dampen artifacts.

![NoData Filled](./images/DTM_NoDataFilled.png)

- Using the above command does a pretty good job of filling the NoData areas.  Image on the left is the filled/smoothed DTM, image on the right is the original DTM with data gaps.  Note that the algorithm does not alter existing values, but only interpolated and smooths the area of missing data.

## Raster Calculator 

Use GDAL raster calculator to difference the DSM and DTM to create a CHM

In [ ]:
 gdal_calc.py -A ./data/FoxIsland_DSM.tif -B ./data/FoxIsland_DTM.tif --outfile ./data/CHM.tif --calc="A-B" --NoDataValue=-9999 --extent intersect



# GDAL Visualizations
- The [gdaldem](https://gdal.org/programs/gdaldem.html) application is a quick and easy way to visualize raster products such as
   - Hillshade
   - Slope
   - Aspect
   - Roughness

In [ ]:
pdal pipeline ./pipelines/DevilsTower.json

In [ ]:
gdaldem hillshade ./data/DevilsTower_Ground.tif ./data/DevilsTower_Ground_HS.tif -z 1 -az 315 -alt 45 

Experiment with some of the different parameters.  For example, try out the "multidirectional" option.  Multidirectional shading is a combination of hillshading illuminated from 225 deg, 270 deg, 315 deg, and 360 deg azimuth.

In [ ]:
gdaldem hillshade ./data/DevilsTower_Ground.tif ./data/DevilsTower_Ground_HSMulti.tif -z 1 -multidirectional

In [ ]:
gdaldem slope ./data/DevilsTower_Ground.tif ./data/DevilsTower_Ground_Slope.tif